In [12]:
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score


%matplotlib inline

### Load Training Data

In [2]:
train_data = pd.read_csv('data.csv')

### 2 cities - São Gonçalo & Vitória

In [3]:
cities = pd.unique(train_data["city"])
print(cities)

['São Gonçalo' 'Vitória']


## São Gonçalo

In [4]:
train_data_Sao = train_data[train_data["city"] == cities[0]]
train_data_Sao.head()

,Unnamed: 0,wsid,wsnm,elvt,lat,lon,inme,city,prov,mdct,...,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
0,0,178,SÃO GONÇALO,237,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 00:00:00,...,29.7,16.8,25.5,10.8,35,58,32,3.2,101,6.5
1,1,178,SÃO GONÇALO,237,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 01:00:00,...,29.9,13.6,29.0,12.2,39,39,35,3.6,94,6.4
2,2,178,SÃO GONÇALO,237,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 02:00:00,...,29.0,14.0,27.4,13.6,44,44,39,2.5,93,6.9
3,3,178,SÃO GONÇALO,237,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 03:00:00,...,27.4,16.9,25.8,14.1,58,58,44,1.7,96,5.8
4,4,178,SÃO GONÇALO,237,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 04:00:00,...,26.3,17.0,25.3,16.4,57,58,56,3.1,110,7.5


#### Data Analyzing

In [5]:
def basic_analyze(df):
    print(df.shape)

    analysis = pd.DataFrame(columns=['col_name', 'nulls', 'uniques', 'dtype'])
    for col in df:
        row = pd.Series({'col_name':col, 
                         'nulls':df[col].isnull().sum(), 
                         'uniques':df[col].unique().size,
                         'dtype':df[col].dtypes})
        analysis = analysis.append(row, ignore_index=True)

    pd.set_option('display.max_rows', 32)
    display(analysis)
    
basic_analyze(train_data_Sao)

(78048, 32)


,col_name,nulls,uniques,dtype
0,Unnamed: 0,0,78048,int64
1,wsid,0,1,int64
2,wsnm,0,1,object
3,elvt,0,1,int64
4,lat,0,1,float64
5,lon,0,1,float64
6,inme,0,1,object
7,city,0,1,object
8,prov,0,1,object
9,mdct,0,78048,object


### Data Cleaning and Preprocessing

#### Drop useless data columns

In [6]:
def dropUselessColumns(df):
    drop_list = []
    for col in df.columns:
        uniques = df[col].unique().size
        if uniques == 1 or uniques == df.shape[0] or col in ['date', 'yr', 'mo', 'da', 'hr']:
            drop_list.append(col)
    
    return df.drop(drop_list, axis=1)
        
pd.set_option('display.max_columns', 30)
train_data_Sao = dropUselessColumns(train_data_Sao)
train_data_Sao.head()

,prcp,stp,smax,smin,gbrd,temp,dewp,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
0,NaN,982.5,982.5,981.3,NaN,29.3,12.1,29.7,16.8,25.5,10.8,35,58,32,3.2,101,6.5
1,NaN,983.2,983.2,982.5,NaN,29.0,13.5,29.9,13.6,29.0,12.2,39,39,35,3.6,94,6.4
2,NaN,983.5,983.5,983.2,NaN,27.4,14.0,29.0,14.0,27.4,13.6,44,44,39,2.5,93,6.9
3,NaN,983.7,983.7,983.4,NaN,25.8,16.9,27.4,16.9,25.8,14.1,58,58,44,1.7,96,5.8
4,NaN,983.7,983.8,983.6,NaN,25.4,16.4,26.3,17.0,25.3,16.4,57,58,56,3.1,110,7.5


#### Fill blanks with zero

In [7]:
def fillBlankByZero(df):
    for col in df.columns:
        df[col].fillna(0, inplace=True)

fillBlankByZero(train_data_Sao)
train_data_Sao.head()

,prcp,stp,smax,smin,gbrd,temp,dewp,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
0,0.0,982.5,982.5,981.3,0.0,29.3,12.1,29.7,16.8,25.5,10.8,35,58,32,3.2,101,6.5
1,0.0,983.2,983.2,982.5,0.0,29.0,13.5,29.9,13.6,29.0,12.2,39,39,35,3.6,94,6.4
2,0.0,983.5,983.5,983.2,0.0,27.4,14.0,29.0,14.0,27.4,13.6,44,44,39,2.5,93,6.9
3,0.0,983.7,983.7,983.4,0.0,25.8,16.9,27.4,16.9,25.8,14.1,58,58,44,1.7,96,5.8
4,0.0,983.7,983.8,983.6,0.0,25.4,16.4,26.3,17.0,25.3,16.4,57,58,56,3.1,110,7.5


#### Find outliers by z-score  (air pressure related fields, like 'stp', 'smax', 'smin' contain erroneous records)

In [99]:
def getOutlierIndices(df):
    df_zscore = (df - df.mean()) / df.std()
    outliers = set()
    outlier_col = []
    for col in df.columns:
        if col == 'prcp':
            continue   
        ol = list(df_zscore[abs(df_zscore[col]) > 3].index)
        if len(ol) > 0:
            outlier_col.append((col, len(ol)))
        outliers.update(ol)
        
    print(outlier_col)
    print(len(outliers))
    return outliers

outliers = getOutlierIndices(train_data_Sao)
train_data_Sao.loc[sorted(outliers)].head()

[('stp', 468), ('smax', 734), ('smin', 643), ('wdsp', 149), ('wdct', 1128), ('gust', 122)]
3146


,prcp,stp,smax,smin,gbrd,temp,dewp,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
25,0.0,985.3,985.3,984.7,0.0,29.3,13.1,30.3,15.1,29.3,13.1,37,41,36,5.4,109,11.5
26,0.0,985.4,985.5,985.3,0.0,28.1,14.0,29.3,14.0,28.1,13.1,42,42,37,6.1,120,11.6
49,0.0,984.5,984.5,983.4,0.0,30.3,16.5,31.6,16.5,30.3,14.8,43,43,36,5.8,119,10.9
73,0.0,982.6,982.6,981.9,0.0,30.6,13.6,31.3,13.7,29.3,12.8,35,38,33,5.4,114,11.9
74,0.0,982.7,4863.8,982.5,0.0,28.3,15.3,30.6,15.3,28.3,13.6,45,45,35,3.4,102,9.6


#### Find rows with all zeros

In [100]:
def getAllZeroIndices(df):
    allZeros = list(df[~((train_data_Sao.T != 0).any())].index)
    print(len(allZeros))
    return allZeros

allZeros = getAllZeroIndices(train_data_Sao)
train_data_Sao.loc[allZeros].head()

10806


,prcp,stp,smax,smin,gbrd,temp,dewp,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0
193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0
194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0


#### Prepare X (Past 3 hours data), Y (Precipitation)

In [101]:
def prepareXYByKHours(data, k, removals):
    X_col_names = []
    for i in range(k, 0, -1):
        for col in data.columns:
            if col == 'prcp':
                continue
            X_col_names.append(col + "(<" + str(i) + "hr)")
    
    df_Y = pd.DataFrame(data=data.loc[k:]['prcp'] , columns=['prcp']).reset_index().drop(['index'], axis=1)
    df_X = pd.DataFrame()
    for start in range(k):
        df_X = pd.concat([df_X, data.iloc[start : -k + start, 1:].reset_index()], axis=1).drop(['index'], axis=1)
    df_X.columns = X_col_names
    
    # drop groups with outliers
    rms = set()
    for rm in list(removals):
        for r in range(max(0, rm - k + 1), min(df_X.shape[0], rm + 1)):
            rms.add(r)
    df_X = df_X.drop(list(rms))
    df_Y = df_Y.drop(list(rms))
        
    return df_X, df_Y

In [102]:
removals = set()
removals.update(outliers, allZeros)
train_X_Sao, train_Y_Sao = prepareXYByKHours(train_data_Sao, 3, removals)

pd.set_option('display.max_columns', 48)
pd.set_option('display.max_rows', 15)
display(pd.concat([train_Y_Sao, train_X_Sao], axis=1))

,prcp,stp(<3hr),smax(<3hr),smin(<3hr),gbrd(<3hr),temp(<3hr),dewp(<3hr),tmax(<3hr),dmax(<3hr),tmin(<3hr),dmin(<3hr),hmdy(<3hr),hmax(<3hr),hmin(<3hr),wdsp(<3hr),wdct(<3hr),gust(<3hr),stp(<2hr),smax(<2hr),smin(<2hr),gbrd(<2hr),temp(<2hr),dewp(<2hr),tmax(<2hr),...,tmin(<2hr),dmin(<2hr),hmdy(<2hr),hmax(<2hr),hmin(<2hr),wdsp(<2hr),wdct(<2hr),gust(<2hr),stp(<1hr),smax(<1hr),smin(<1hr),gbrd(<1hr),temp(<1hr),dewp(<1hr),tmax(<1hr),dmax(<1hr),tmin(<1hr),dmin(<1hr),hmdy(<1hr),hmax(<1hr),hmin(<1hr),wdsp(<1hr),wdct(<1hr),gust(<1hr)
0,0.0,982.5,982.5,981.3,0.000,29.3,12.1,29.7,16.8,25.5,10.8,35,58,32,3.2,101,6.5,983.2,983.2,982.5,0.000,29.0,13.5,29.9,...,29.0,12.2,39,39,35,3.6,94,6.4,983.5,983.5,983.2,0.000,27.4,14.0,29.0,14.0,27.4,13.6,44,44,39,2.5,93,6.9
1,0.0,983.2,983.2,982.5,0.000,29.0,13.5,29.9,13.6,29.0,12.2,39,39,35,3.6,94,6.4,983.5,983.5,983.2,0.000,27.4,14.0,29.0,...,27.4,13.6,44,44,39,2.5,93,6.9,983.7,983.7,983.4,0.000,25.8,16.9,27.4,16.9,25.8,14.1,58,58,44,1.7,96,5.8
2,0.0,983.5,983.5,983.2,0.000,27.4,14.0,29.0,14.0,27.4,13.6,44,44,39,2.5,93,6.9,983.7,983.7,983.4,0.000,25.8,16.9,27.4,...,25.8,14.1,58,58,44,1.7,96,5.8,983.7,983.8,983.6,0.000,25.4,16.4,26.3,17.0,25.3,16.4,57,58,56,3.1,110,7.5
3,0.0,983.7,983.7,983.4,0.000,25.8,16.9,27.4,16.9,25.8,14.1,58,58,44,1.7,96,5.8,983.7,983.8,983.6,0.000,25.4,16.4,26.3,...,25.3,16.4,57,58,56,3.1,110,7.5,983.7,983.8,983.6,0.000,23.8,16.2,25.4,16.4,23.8,16.0,62,62,57,2.0,99,6.8
4,0.0,983.7,983.8,983.6,0.000,25.4,16.4,26.3,17.0,25.3,16.4,57,58,56,3.1,110,7.5,983.7,983.8,983.6,0.000,23.8,16.2,25.4,...,23.8,16.0,62,62,57,2.0,99,6.8,983.7,983.7,983.6,0.000,22.0,16.7,23.8,16.7,22.0,16.2,72,72,62,1.3,93,4.9
5,0.0,983.7,983.8,983.6,0.000,23.8,16.2,25.4,16.4,23.8,16.0,62,62,57,2.0,99,6.8,983.7,983.7,983.6,0.000,22.0,16.7,23.8,...,22.0,16.2,72,72,62,1.3,93,4.9,984.6,984.6,983.7,0.000,19.7,17.4,22.0,17.8,19.5,16.6,86,89,72,0.5,157,2.8
6,0.0,983.7,983.7,983.6,0.000,22.0,16.7,23.8,16.7,22.0,16.2,72,72,62,1.3,93,4.9,984.6,984.6,983.7,0.000,19.7,17.4,22.0,...,19.5,16.6,86,89,72,0.5,157,2.8,985.7,985.7,984.6,0.000,18.3,17.3,19.7,17.3,18.3,16.9,93,94,85,0.0,141,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78038,0.0,986.9,988.0,986.9,3389.313,31.4,13.3,32.1,15.1,29.7,13.2,33,41,33,4.7,98,10.4,985.7,986.9,985.7,3025.471,33.1,13.1,33.2,...,31.4,12.2,30,33,29,4.4,103,9.5,984.2,985.7,984.2,3531.766,34.3,11.7,34.7,13.3,32.5,10.8,25,31,24,4.4,75,10.3
78039,0.0,985.7,986.9,985.7,3025.471,33.1,13.1,33.2,13.8,31.4,12.2,30,33,29,4.4,103,9.5,984.2,985.7,984.2,3531.766,34.3,11.7,34.7,...,32.5,10.8,25,31,24,4.4,75,10.3,982.9,984.3,982.9,3078.407,35.2,11.1,35.3,12.1,33.7,10.8,23,26,23,4.1,87,8.8


### Train models (logistic regression + linear regression)

In [103]:
raining_idx = train_Y_Sao['prcp'] > 0

#### A logistic regression model that predicts whether it will rain in the next hour

In [104]:
%%time 

logistic = LogisticRegression()
scores = cross_val_score(logistic, train_X_Sao, raining_idx, cv=5)
print("Logistic Regression: 5-fold Cross-Validation: Mean Accuracy: %f" % (scores.mean()))

Logistic Regression: 5-fold Cross-Validation: Mean Accuracy: 0.961620
CPU times: user 32.6 s, sys: 558 ms, total: 33.2 s
Wall time: 31.8 s


#### A linear regression model predicting how much it will rain

In [117]:
%%time

linear = LinearRegression()
scores = cross_val_score(linear, train_X_Sao[raining_idx], train_Y_Sao[raining_idx], cv=5)
print(scores)
print("Linear Regression: 5-fold Cross-Validation: R2 score: %f" % (scores.mean()))

[-2.27852912  0.02458496  0.01585072  0.02514614  0.00799988]
Linear Regression: 5-fold Cross-Validation: R2 score: -0.440989
CPU times: user 147 ms, sys: 83.6 ms, total: 230 ms
Wall time: 45.8 ms
